# Classification de prénoms en genre (masculin/féminin) 🇫🇷

[Xiaoou WANG](https://https://scholar.google.fr/citations?user=vKAMMpwAAAAJ&hl=en)

Cet exemple très simple issu du [livre](https://www.amazon.fr/Natural-Language-Processing-Python-Steven/dp/0596516495/ref=sr_1_11?__mk_fr_FR=%C3%85M%C3%85%C5%BD%C3%95%C3%91&dchild=1&keywords=nltk&qid=1616280931&sr=8-11) de Steven Bird vous donne une idée sur à quoi ressemble le boulot d'un ingénieur junior en Machine Learning.

La tâche consiste à entraîner un classifieur bayésien pour prédire le genre d'un prénom.


## Sélection de features

On commence par prendre la dernière lettre d'un prénom comme feature et la stocker dans un dictionnaire.

In [3]:
#! creation de last latter comme feature

import nltk
from nltk.corpus import names
import random
random.seed(13)
def gender_features(word):
     return {'last_letter': word[-1]}

print("La dernière lettre du pronom Shrek est")
gender_features('Shrek')

La dernière lettre du prnom Shrek est


{'last_letter': 'k'}

## Mise en forme du corpus

Le corpus provient de `nltk`. Ici on crée une liste de tuples grâce à quelques méthodes intégrées dans `nltk.corpus`. Notez l'emploi de list comprehension ici pour rendre le code plus concis tout en gardant la lisibilité.

In [6]:
#! creation de datasets

labeled_names = ([(name, 'male') for name in names.words('male.txt')] +
 [(name, 'female') for name in names.words('female.txt')])

random.shuffle(labeled_names)
print("Un échantillon du corpus")
labeled_names[:10]

Un échantillon du corpus


[('Mariam', 'female'),
 ('Marjorie', 'female'),
 ('Jasmin', 'female'),
 ('Welbie', 'male'),
 ('Modesty', 'female'),
 ('Kanya', 'female'),
 ('Michale', 'male'),
 ('Antonina', 'female'),
 ('Beulah', 'female'),
 ('Hazel', 'female')]

## Création des corpus train/test

Ici on applique à la fonction de la `section 1` à tous les noms du corpus. Les 500 premiers samples sont mis à l'écart pour servir de test.

In [10]:
#! creation de paire feature/label

featuresets = [(gender_features(n), gender) for (n, gender) in labeled_names]

In [11]:
#! creation de train et test

train_set, test_set = featuresets[500:], featuresets[:500]

## Première classification

La précision est autour de 75.2%. On liste les features les plus utiles pour étudier quels sont les problèmes potentiels. Le likely ratio `male : female` signifie la probabilité exacte qu'un prénom particulier soit masculin/féminin en fonction de sa dernière lettre (donc feature).

In [12]:
classifier = nltk.NaiveBayesClassifier.train(train_set)
#! classify
classifier.classify(gender_features('Neo'))
classifier.classify(gender_features('Trinity'))
print(nltk.classify.accuracy(classifier, test_set))
classifier.show_most_informative_features(5)
#! see likely ratio

0.752
Most Informative Features
             last_letter = 'k'              male : female =     45.8 : 1.0
             last_letter = 'a'            female : male   =     33.0 : 1.0
             last_letter = 'f'              male : female =     15.3 : 1.0
             last_letter = 'p'              male : female =     11.2 : 1.0
             last_letter = 'v'              male : female =     10.5 : 1.0


## Ajout de features et problème d'overfitting

Si vous ajoutez trop de features, le modèle risque d'être trop adapté à tes données et se généralise mal sur des données non vues. Cela s'appelle `overfitting` et survient souvent quand le corpus est petit, ce qui est le cas ici.

## Let's add features

Dans un premier temps, nous allons essayer d'ajouter plein de features.

Examinons la fonction `gender_features2`. Les features sont :

* La première et la dernière lettre
* Un booléen indiquant si une lettre de l'ensemble a-z est présent dans le prénom
* Un integer indiquant le nombre d'occurrences de cette lettre

Donc clairement nous y mettons tout le paquet...

In [13]:
#! add features
def gender_features2(name):
    features = {"first_letter": name[0].lower(), "last_letter": name[-1].lower()}
    for letter in 'abcdefghijklmnopqrstuvwxyz':
        features["count({})".format(letter)] = name.lower().count(letter)
        features["has({})".format(letter)] = (letter in name.lower())
    return features

demo = gender_features2("john")
random.sample(demo.items(),5)

[('count(i)', 0),
 ('count(h)', 1),
 ('count(x)', 0),
 ('has(a)', False),
 ('has(h)', True)]

## La précision augmente

Avec ce nouveau featureset, la précision est montée de 75.2% à 77.4%.

In [14]:
featuresets = [(gender_features2(n), gender) for (n, gender) in labeled_names]
train_set, test_set = featuresets[500:], featuresets[:500]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

#! 0.752 vs 0.774

0.774


## Feature engineering via l'analyse des erreurs

Ici nous expliquons le processus de feature engineering qui consiste à analyser les erreurs de la machine sur la base desquelles on filtre/supprime/créé des features.

L'intelligence artificielle n'est finalement pas si artificielle, non ? (Bon j'avoue que c'est pas du deep learning, mais quand même)

Notons ici la création de 'devset'. Cette répartition en 3 sets est canonique en Machine Learning. On utilise le train pour entraîner le modèle, le devset pour ajuster ce modèle. Enfin le test ne doit être utilisé que pour l'évaluation finale.

La précision initiale (avant le feature engineering) est donc 76%.

In [15]:
train_names = labeled_names[1500:]
devtest_names = labeled_names[500:1500]
test_names = labeled_names[:500]

In [16]:
#! use devtest
train_set = [(gender_features(n), gender) for (n, gender) in train_names]
devtest_set = [(gender_features(n), gender) for (n, gender) in devtest_names]
test_set = [(gender_features(n), gender) for (n, gender) in test_names]
classifier = nltk.NaiveBayesClassifier.train(train_set)

print(nltk.classify.accuracy(classifier, devtest_set))

0.76


## Premières hypothèses sur les erreurs

Analysons les erreurs affichées ci-dessous.

Les pronoms terminés par `yn` tendent à être féminins, alors que ceux terminés par `n` tendent à être masculins. Du coup deux règles serait meilleures qu'une seule.

Ca semble être le même principe pour les pronoms terminés par `h` qui sont principalement féminins et ceux terminés par `ch` qui ont tendance à être masculins

In [17]:
errors = []
for (name, tag) in devtest_names:
     guess = classifier.classify(gender_features(name))
     if guess != tag:
         errors.append( (tag, guess, name) )

for (tag, guess, name) in sorted(errors):
     print('correct={:<8} guess={:<8s} name={:<30}'.format(tag, guess, name))

correct=female   guess=male     name=Aeriel                        
correct=female   guess=male     name=Aeriell                       
correct=female   guess=male     name=Allis                         
correct=female   guess=male     name=Allsun                        
correct=female   guess=male     name=Allyn                         
correct=female   guess=male     name=Allys                         
correct=female   guess=male     name=Amargo                        
correct=female   guess=male     name=Amber                         
correct=female   guess=male     name=Anne-Mar                      
correct=female   guess=male     name=Aurel                         
correct=female   guess=male     name=Avril                         
correct=female   guess=male     name=Barb                          
correct=female   guess=male     name=Beatriz                       
correct=female   guess=male     name=Beilul                        
correct=female   guess=male     name=Calypso    

## Intégration des nouveaux features dans le classifieur

Il semble bénéfique d'ajuster nos features en incluant les deux dernières lettres.

Et youpi ! La précision est montée de 76% à 78.1%. C'est pas mal non ?

In [19]:
def gender_features(word):
     return {'suffix1': word[-1:],
             'suffix2': word[-2:]}

train_set = [(gender_features(n), gender) for (n, gender) in train_names]
devtest_set = [(gender_features(n), gender) for (n, gender) in devtest_names]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, devtest_set))

# 0.76 -> 0.781

0.781


## Importance de nouveaux splits train/dev

Nous pouvons donc réitérer ce processus d'analyse d'erreurs et de feature engineering jusqu'à obtenir une performance satisfaisante.

Attention !:D

Il vaut mieux faire un nouveau split train/dev à chaque fois qu'on intègre/supprime des features pour éviter l'overfitting.


## Conclusions

Bravo d'avoir fini l'article !

Ce qu'il faut retenir :

1. Le machine learning traditionnel repose pas mal sur l'analyse humaine. Comme vous avez vu ici, l'analyse des erreurs de classification aide beaucoup l'intelligence "artificielle".

2. Il est important de faire un split train/dev/test pour éviter que le modèle soit overfitted. Dans la même ligne de pensée il est aussi conseillé de garder un nombre raisonnable de features.

3. Vous l'aurez compris. L'analyse d'erreurs (feature engineering) et le compromis entre performance et généralisabilité font du machine learning un art qui nécessite un savoir-faire qui s'acquiert au fil des ans.

[Reference](https://www.nltk.org/book/ch06.html)
